In [5]:
import importlib

import apollo.providers.price_data_provider as pdp
from apollo.settings import (
    END_DATE,
    FREQUENCY,
    MAX_PERIOD,
    START_DATE,
    TICKER,
    VIX_TICKER,
)

importlib.reload(pdp);

In [6]:
vix_price_data_provider = pdp.PriceDataProvider(
    ticker=str(VIX_TICKER),
    frequency=str(FREQUENCY),
    start_date=str(START_DATE),
    end_date=str(END_DATE),
    max_period=bool(MAX_PERIOD),
)

vix_price_data = vix_price_data_provider.get_price_data()

price_data_provider = pdp.PriceDataProvider(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    start_date=str(START_DATE),
    end_date=str(END_DATE),
    max_period=bool(MAX_PERIOD),
)

price_data = price_data_provider.get_price_data()

           ticker  adj close   adj high    adj low   adj open  adj volume  \
date                                                                        
1990-01-02   ^VIX  17.240000  17.240000  17.240000  17.240000         0.0   
1990-01-03   ^VIX  18.190001  18.190001  18.190001  18.190001         0.0   
1990-01-04   ^VIX  19.219999  19.219999  19.219999  19.219999         0.0   
1990-01-05   ^VIX  20.110001  20.110001  20.110001  20.110001         0.0   
1990-01-08   ^VIX  20.260000  20.260000  20.260000  20.260000         0.0   
...           ...        ...        ...        ...        ...         ...   
2024-08-05   ^VIX  38.570000  65.730003  23.389999  23.389999         0.0   
2024-08-06   ^VIX  27.709999  34.770000  24.020000  33.709999         0.0   
2024-08-07   ^VIX  27.850000  29.760000  21.969999  24.770000         0.0   
2024-08-08   ^VIX  23.790001  29.469999  23.360001  28.340000         0.0   
2024-08-09   ^VIX  20.370001  24.520000  20.260000  23.780001         0.0   